In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

In [31]:
kindle_df = pd.read_csv('kindle.csv')
kindle_df.head()

,review_title,product_review,product_rating,reviewer
0,"Works fine, replaces a Gen 3.",\n If you own an earlier Paperwhite and it's ...,3.0 out of 5 stars,E. Hinrichsen II
1,Pinhole Bright Spots and Other Issues,"\n Updated Review on Oct 6, 2019:",4.0 out of 5 stars,Sir Mike
2,Battery life is awful,"\n I love Kindle, have always had one since t...",3.0 out of 5 stars,Ship210
3,A Review From 2 Perspectives,\n I feel I need to approach this review from...,5.0 out of 5 stars,Rustic
4,It’s waterproof. Everything else is worse.,\n It’s infuriating that Amazon went to marke...,1.0 out of 5 stars,D. Bier


In [32]:
# \n represents the new line
# let's replace it with 
kindle_df['product_review'] = kindle_df['product_review'].str.replace('\n', '')
kindle_df.head()

,review_title,product_review,product_rating,reviewer
0,"Works fine, replaces a Gen 3.",If you own an earlier Paperwhite and it's st...,3.0 out of 5 stars,E. Hinrichsen II
1,Pinhole Bright Spots and Other Issues,"Updated Review on Oct 6, 2019:",4.0 out of 5 stars,Sir Mike
2,Battery life is awful,"I love Kindle, have always had one since the...",3.0 out of 5 stars,Ship210
3,A Review From 2 Perspectives,I feel I need to approach this review from 2...,5.0 out of 5 stars,Rustic
4,It’s waterproof. Everything else is worse.,It’s infuriating that Amazon went to market ...,1.0 out of 5 stars,D. Bier


In [82]:
data = kindle_df['product_review'][:2000]
data

0         If you own an earlier Paperwhite and it's st...
1                          Updated Review on Oct 6, 2019:
2         I love Kindle, have always had one since the...
3         I feel I need to approach this review from 2...
4         It’s infuriating that Amazon went to market ...
                              ...                        
1995      I am so disappointed in the Kindle Ereaders....
1996      Call me crazy, but I bought a Kindle to supp...
1997      I bought this for the new Bluetooth feature ...
1998      I rarely use my new Kindle Paperwhite, so so...
1999      I like the Paperwhite but I still miss my 3r...
Name: product_review, Length: 2000, dtype: object

In [83]:
print("product_review: ",kindle_df['product_review'].isnull().values.any())
nan_values = kindle_df[kindle_df['product_review'].isnull()]
nan_values

product_review:  False


,review_title,product_review,product_rating,reviewer


In [84]:
# if rating is positive, fill with good
# if rating is negative, fill with bad
# if rating is neutral, fill with average

# kindle_df['product_review'].fillna('bad')
kindle_df['product_review'] = kindle_df.product_review.replace(np.nan, 'bad', regex=True)
print("product_review: ",kindle_df['product_review'].isnull().values.any())
kindle_df

product_review:  False


,review_title,product_review,product_rating,reviewer
0,"Works fine, replaces a Gen 3.",If you own an earlier Paperwhite and it's st...,3.0 out of 5 stars,E. Hinrichsen II
1,Pinhole Bright Spots and Other Issues,"Updated Review on Oct 6, 2019:",4.0 out of 5 stars,Sir Mike
2,Battery life is awful,"I love Kindle, have always had one since the...",3.0 out of 5 stars,Ship210
3,A Review From 2 Perspectives,I feel I need to approach this review from 2...,5.0 out of 5 stars,Rustic
4,It’s waterproof. Everything else is worse.,It’s infuriating that Amazon went to market ...,1.0 out of 5 stars,D. Bier
...,...,...,...,...
9996,My first Kindle - Love it!,"I've never used a Kindle, or any e-reader befo...",5.0 out of 5 stars,AngieF
9997,Amazon con job,"Can no longer ""Remove this title"" from my Kind...",3.0 out of 5 stars,thom flora
9998,A necessary evil...,I bought the Kindle Paperwhite so I could down...,4.0 out of 5 stars,Sylvia
9999,New and imporved,Smaller and lighter (green is the new one and ...,5.0 out of 5 stars,kattface6


In [85]:
len(data)

2000

In [86]:
reviews = [h for h in data ]

In [87]:
def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii", "ignore")
    return txt
corpus = [clean_text(x) for x in reviews]

In [88]:
corpus[45]

'  i purchased two of your ereaders and neither one came with a charge adapter   how can you expect the customer to charge your reader without the necessary equipment   this necessary piece of electronics should automatically be included   not happy that my granddaughter and i couldnt use our new ereaders until we purchased the charge cords '

# Model

In [89]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku
import re, string 

In [90]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

In [91]:
total_words

8182

In [92]:
input_sequences = []

for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n = token_list[:i+1]
        input_sequences.append(n)

In [93]:
len(input_sequences)

206455

In [94]:
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [95]:
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

label = ku.to_categorical(label, num_classes=total_words)

In [96]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, GRU, Dropout, Bidirectional, SpatialDropout1D

In [97]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len))
model.add(Bidirectional(GRU(150, return_sequences=True)))
model.add(Dropout(0.2))
model.add(GRU(100))
# model.add(Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 616, 100)          818200    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 616, 300)          226800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 616, 300)          0         
_________________________________________________________________
gru_3 (GRU)                  (None, 100)               120600    
_________________________________________________________________
dense_1 (Dense)              (None, 8182)              826382    
Total params: 1,991,982
Trainable params: 1,991,982
Non-trainable params: 0
_________________________________________________________________
None


In [98]:
model.fit(predictors, label, epochs=50, verbose=1)

Epoch 1/50
6452/6452 [==============================] - 5337s 827ms/step - loss: 6.2097 - accuracy: 0.0653
Epoch 2/50
1950/6452 [========>.....................] - ETA: 1:01:06 - loss: 5.2458 - accuracy: 0.1305

KeyboardInterrupt: 